In [1]:
# Data stuff
import numpy as np
import pandas as pd

# Visual stuff
from IPython.display import display
# Configs 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

# Random state seed
rand_state=42

# ML stuff
import sklearn
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import shap

In [2]:
dataset = pd.read_csv('data_gen/dataset_mean_entire.csv')
# Ensure correct types
dataset = dataset.astype(dtype={
    'age':float,
    'gender':int,
    'expired':int,
    'P-glucose':float,
    'blood_pressure_systoliskt':float,
    'blood_pressure_diastoliskt':float,
    'BMI':float
    
})
# Order ints(categorical variables, first) and floats(number variables, last)
dataset = dataset[[
    'gender',
    'I109',
    'E119',
    'E669',
    'I259',
    'I252',
    'I209',
    'E660',
    'E118',
    'I639',
    'E113',
    'expired',
    'age',
    'P-glucose',
    'blood_pressure_systoliskt',
    'blood_pressure_diastoliskt',
    'BMI',
]]

In [3]:
# Generate data set withouth categories(all numbers)
Y_no_cat = dataset.expired.values
X_no_cat = dataset.drop(columns=['expired']).values

# dataset as is, but target variable dropped(hospital expire flag)
dataset_no_target = dataset.drop(columns=['expired'])

# Feture names and categorical feature names
feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() + dataset_no_target.select_dtypes(exclude='int').columns.values.tolist() 
cat_feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() 

# Generate data set with categories(int type required)
dataframe_int_list = dataset_no_target.select_dtypes(include='int').values.tolist()
dataframe_no_int_list = dataset_no_target.select_dtypes(exclude='int').values.tolist()
Y = dataset.expired.values.tolist()
X = []
for i,v in enumerate(dataframe_int_list):
    X = X + [v+dataframe_no_int_list[i]]

# Generate categorical feature indicies
cat_features_indices=list(range(0,len(dataframe_int_list[0])))

In [4]:
# Function that w
def strat_cv_it(classifier, params, uses_cat, param_comb ):
    folds = 3
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = rand_state)
    
    # Perform fit and scoring
    if uses_cat:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X, Y),random_state = rand_state, refit=True, verbose=3, error_score=0.0 )
        random_search.fit(X, Y)     
    else:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X_no_cat, Y_no_cat),random_state = rand_state, refit=True, verbose=3,error_score=0.0)
        random_search.fit(X_no_cat, Y_no_cat)
    
    # Display results and return best model
    display(random_search.best_score_)
    display(random_search.best_params_)
    display(pd.DataFrame(random_search.cv_results_))
    return random_search.best_estimator_

# Scale of negative class to the positive class(#survived/#died)
scale_pos_weight_min = int( (dataset[dataset.expired==0].shape[0] / dataset[dataset.expired==1].shape[0]) )

# XGboost

In [5]:
xgb_classifier = xgb.XGBClassifier(objective = "binary:logistic",random_state=rand_state)
xgb_params = {
        'learning_rate': (0.01, 0.05,0.1),
        'min_child_weight': [3, 5, 10],
        'gamma': [0.5, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': list(range(5,30)),
        'scale_pos_weight':  list(range(scale_pos_weight_min,3*scale_pos_weight_min))
}
model = strat_cv_it(xgb_classifier ,xgb_params,False,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  8.9min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:  9.7min finished


0.8787483021790136

{'subsample': 0.6,
 'scale_pos_weight': 25,
 'min_child_weight': 10,
 'max_depth': 28,
 'learning_rate': 0.01,
 'gamma': 2,
 'colsample_bytree': 0.8}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_scale_pos_weight,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,4.047445,0.208407,0.023716,0.002214,1,22,5,7,0.01,1.5,1,"{'subsample': 1.0, 'scale_pos_weight': 22, 'mi...",0.858815,0.875710,0.858422,0.864316,0.008059,22,0.941209,0.941596,0.938968,0.940591,0.001159
1,11.978372,1.526559,0.046565,0.012939,1,25,10,20,0.1,2,1,"{'subsample': 1.0, 'scale_pos_weight': 25, 'mi...",0.851995,0.841649,0.842795,0.845479,0.004631,44,0.994705,0.994124,0.993016,0.993949,0.000701
2,11.576724,2.546015,0.047226,0.014575,0.8,27,10,12,0.1,1.5,1,"{'subsample': 0.8, 'scale_pos_weight': 27, 'mi...",0.861424,0.855299,0.840710,0.852478,0.008688,38,0.988205,0.988962,0.989911,0.989026,0.000698
3,6.666098,1.357407,0.050426,0.016120,1,14,3,6,0.1,5,0.8,"{'subsample': 1.0, 'scale_pos_weight': 14, 'mi...",0.870892,0.863347,0.846016,0.860085,0.010414,29,0.979923,0.980949,0.978918,0.979930,0.000829
4,54.689945,4.733557,0.070511,0.013483,1,29,10,20,0.1,0.5,1,"{'subsample': 1.0, 'scale_pos_weight': 29, 'mi...",0.848075,0.839862,0.843207,0.843715,0.003372,45,0.995568,0.994812,0.993896,0.994759,0.000684
5,55.936784,7.760502,0.056787,0.002807,0.6,26,5,17,0.01,0.5,1,"{'subsample': 0.6, 'scale_pos_weight': 26, 'mi...",0.873131,0.870193,0.868888,0.870737,0.001775,9,0.956901,0.959667,0.957055,0.957874,0.001269
6,35.666354,5.107845,0.048709,0.006387,0.8,25,10,9,0.01,0.5,0.8,"{'subsample': 0.8, 'scale_pos_weight': 25, 'mi...",0.884344,0.873038,0.871565,0.876316,0.005709,2,0.950753,0.952215,0.952303,0.951757,0.000711
7,44.450126,6.905370,0.051141,0.008446,0.6,29,10,17,0.01,2,1,"{'subsample': 0.6, 'scale_pos_weight': 29, 'mi...",0.883048,0.870186,0.875634,0.876290,0.005271,3,0.942398,0.945167,0.943975,0.943847,0.001134
8,47.392796,1.663571,0.052239,0.012918,0.6,16,3,16,0.05,2,0.8,"{'subsample': 0.6, 'scale_pos_weight': 16, 'mi...",0.861400,0.853468,0.861444,0.858771,0.003750,30,0.991428,0.991154,0.990457,0.991013,0.000408
9,28.200304,7.109790,0.057420,0.014863,1,25,3,6,0.05,0.5,1,"{'subsample': 1.0, 'scale_pos_weight': 25, 'mi...",0.869561,0.872321,0.848503,0.863462,0.010637,25,0.970952,0.969230,0.967231,0.969137,0.001521


In [ ]:
shap_values = explainer.shap_values( X_no_cat)
shap.summary_plot(shap_values,X_no_cat ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names, interaction_index='gender', show=False)
        
    # plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')

# LightGBM

In [7]:
LGB_classifier = lgb.LGBMClassifier()
LGB_params = {
             'num_leaves': [1,5,8,10,15,20,35,40], 
             'min_child_samples': [1,5,10,20,50,100,200,300,400,500], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': [0.2,0,4,0.5, 0.6, 0.8, 1.0],
             'colsample_bytree': [0.6, 0.8, 1.0],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             'scale_pos_weight':  list(range(scale_pos_weight_min,3*scale_pos_weight_min))

}
model = strat_cv_it(LGB_classifier ,LGB_params,True,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   14.3s
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:   18.0s finished


0.8868821085994819

{'subsample': 0.8,
 'scale_pos_weight': 17,
 'reg_lambda': 10,
 'reg_alpha': 50,
 'num_leaves': 8,
 'min_child_weight': 100.0,
 'min_child_samples': 400,
 'colsample_bytree': 1.0}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_scale_pos_weight,param_reg_lambda,param_reg_alpha,param_num_leaves,param_min_child_weight,param_min_child_samples,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.008009,0.000625,0.000000,0.000000,0,21,0,10,15,0.001,10,1,"{'subsample': 0, 'scale_pos_weight': 21, 'reg_...",0.000000,0.000000,0.000000,0.000000,0.000000,33,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.061190,0.032000,0.006716,0.000943,0.2,14,0,5,10,1e-05,500,1,"{'subsample': 0.2, 'scale_pos_weight': 14, 're...",0.890680,0.875810,0.877505,0.881331,0.006647,9,0.891961,0.898524,0.900811,0.897099,0.003751
2,0.006960,0.000121,0.000000,0.000000,0.2,26,5,100,1,0.1,400,0.8,"{'subsample': 0.2, 'scale_pos_weight': 26, 're...",0.000000,0.000000,0.000000,0.000000,0.000000,33,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.099270,0.017004,0.008631,0.001364,0.8,22,0,10,20,1e-05,20,0.6,"{'subsample': 0.8, 'scale_pos_weight': 22, 're...",0.879262,0.865616,0.865439,0.870106,0.006475,21,0.950881,0.953419,0.949803,0.951368,0.001516
4,0.476349,0.021362,0.011891,0.001518,1,10,5,0,40,1e-05,1,1,"{'subsample': 1.0, 'scale_pos_weight': 10, 're...",0.866214,0.848650,0.846021,0.853628,0.008964,26,0.992241,0.993719,0.992462,0.992807,0.000651
5,0.123758,0.047514,0.013660,0.002286,0.8,12,100,0.1,10,0.01,20,0.8,"{'subsample': 0.8, 'scale_pos_weight': 12, 're...",0.888661,0.867888,0.875050,0.877200,0.008616,13,0.917983,0.925548,0.923059,0.922197,0.003148
6,0.050441,0.015827,0.007818,0.000567,0.6,19,100,0,15,0.01,500,1,"{'subsample': 0.6, 'scale_pos_weight': 19, 're...",0.892967,0.875245,0.878053,0.882089,0.007777,5,0.891276,0.896689,0.898144,0.895370,0.002955
7,0.125643,0.033320,0.008972,0.002180,0.8,25,100,1,15,1,50,0.8,"{'subsample': 0.8, 'scale_pos_weight': 25, 're...",0.886864,0.866639,0.875073,0.876192,0.008295,14,0.922647,0.927580,0.925038,0.925088,0.002014
8,0.054057,0.013039,0.006494,0.000600,0.6,13,20,0.1,5,0.1,500,1,"{'subsample': 0.6, 'scale_pos_weight': 13, 're...",0.891910,0.874953,0.878118,0.881661,0.007362,8,0.893168,0.899232,0.900157,0.897519,0.003100
9,0.210925,0.058935,0.010676,0.004038,0.2,26,5,7,35,0.001,1,0.8,"{'subsample': 0.2, 'scale_pos_weight': 26, 're...",0.871305,0.859732,0.857983,0.863007,0.005911,24,0.975422,0.975523,0.976391,0.975779,0.000435


In [ ]:
shap_values = explainer.shap_values(dataset.drop(columns=['expired']))
shap.summary_plot(shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, interaction_index='gender', show=False)
        
  #  plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')